<a href="https://colab.research.google.com/github/rahulchirra/torrent-downloder/blob/main/100mbps_79.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install libtorrent
!apt install python3-libtorrent


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libtorrent-rasterbar2.0
The following NEW packages will be installed:
  libtorrent-rasterbar2.0 python3-libtorrent
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,098 kB of archives.
After this operation, 7,907 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtorrent-rasterbar2.0 amd64 2.0.5-5 [1,501 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-libtorrent amd64 2.0.5-5 [597 kB]
Fetched 2,098 kB in 1s (1,858 kB/s)
Selecting previously unselected package libtorrent-rasterbar2.0:amd64.
(Reading database ... 123595 files and directories currently installed.)
Preparing to unpack .../libtorrent-rasterbar2.0_2.0.5-5_amd64.deb ...
Unpacking libtorrent-rasterbar2.0:amd64 (2.0.5-5) ...
Selecting previously unselected package python3-li

In [5]:
import libtorrent as lt
import time
from IPython.display import display
import ipywidgets as widgets
from google.colab import files, drive

# Mount Google Drive
drive.mount('/content/drive')

# Initialize the session
ses = lt.session()
ses.listen_on(6881, 6891)

# Set maximum download speed (100 Mbps = 100 * 1024 * 1024 bytes/s)
# The original value was too large, causing an overflow error.
# Libtorrent expects the rate limit in bytes per second.
max_download_speed = 100 * 1024 * 1024  # 100 Mbps
ses.set_download_rate_limit(max_download_speed)

downloads = []


# Add From Magnet Link
while True:
    params = {"save_path": "/content/drive/My Drive/Torrents/"}  # Change this to a Google Drive directory
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

# Start Download
state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            # Convert download rate from kB/s to MB/s
            download_rate_mbps = s.download_rate / 1024 / 1024  # Convert from bytes to megabytes
            bar.description = " ".join(
                [
                    download.name(),
                    "{:.2f} MB/s".format(download_rate_mbps),
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close()  # This might not fully work in Colab
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

<ipython-input-5-60a29f3abe46>:12: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)
<ipython-input-5-60a29f3abe46>:18: DeprecationWarning: set_download_rate_limit() is deprecated
  ses.set_download_rate_limit(max_download_speed)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:52CBF4739C47538A0E08296AC87512B28CA280A6&dn=Harry+Potter+and+the+Chamber+of+Secrets+%282002%29+%5B2160p%5D+%5BYTS.MX%5D&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Fopen.tracker.cl%3A1337%2Fannounce&tr=udp%3A%2F%2Fp4p.arenabg.com%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce&tr=udp%3A%2F%2Ftracker.dler.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Fipv4.tracker.harry.lu%3A80%2Fannounce&tr=https%3A%2F%2Fopentracker.i2p.rocks%3A443%2Fannounce


<ipython-input-5-60a29f3abe46>:30: DeprecationWarning: add_magnet_uri() is deprecated
  lt.add_magnet_uri(ses, magnet_link, params)


Enter Magnet Link Or Type Exit: exit


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

<ipython-input-5-60a29f3abe46>:59: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
<ipython-input-5-60a29f3abe46>:66: DeprecationWarning: name() is deprecated
  download.name(),
<ipython-input-5-60a29f3abe46>:78: DeprecationWarning: name() is deprecated
  print(download.name(), "complete")


Harry Potter And The Chamber Of Secrets (2002) [2160p] [4K] [BluRay] [5.1] [YTS.MX] complete
